<a href="https://colab.research.google.com/github/Kommmi/Quantum-Dynamical-Systems/blob/main/GQS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!rm -rf Quantum-Dynamical-Systems
!git clone https://github.com/Kommmi/Quantum-Dynamical-Systems.git

# Move into the repo
%cd Quantum-Dynamical-Systems

# Install dependencies
!pip install -r requirements.txt

# Install the package in editable mode
!pip install -e .

from IPython.display import clear_output
clear_output()

import numpy as np
import matplotlib.pyplot as plt
from gqs.states import Initial_state
from gqs.dynamics import Hamiltonian_QK, floquet_operator_from_H
from gqs.distances import Quantum_EMD
from gqs.gqs import GQS_Bloch_Sphere_chi

print("Module ready to go :)")


Module ready to go :)


In [ ]:
nqubit = 4
site_a = 0
theta, phi = 0.8, 0.2
eps = 1e-2
theta_p, phi_p = theta + eps, phi

# Kicked-top parameters
kappa = 3.0
j = nqubit / 2
tau = 1.0
T = 25  # number of kicks / discrete steps

psi0 = Initial_state(nqubit, theta, phi)
psi0_p = Initial_state(nqubit, theta_p, phi_p)

print("State shapes:", psi0.shape, psi0_p.shape)
print("Norms:", np.vdot(psi0, psi0).real, np.vdot(psi0_p, psi0_p).real)

State shapes: (16,) (16,)
Norms: 1.0000000000000002 0.9999999999999994


In [ ]:
H = Hamiltonian_QK(nqubit, kappa=kappa, j=j)
U = floquet_operator_from_H(H, nqubit=nqubit, tau=tau)

def evolve(U, psi, T):
    out = [psi]
    for _ in range(T):
        psi = U @ psi
        out.append(psi)
    return out

traj = evolve(U, psi0, T)
traj_p = evolve(U, psi0_p, T)

print("Generated trajectories of length", len(traj))


TypeError: Hamiltonian_QK() got an unexpected keyword argument 'j'

In [ ]:
def gqs_for_site(psi_t, site_a, nqubit):
    # Reconstruct chi_j and lambda_j by reshaping the state into (system, environment).
    # For a single qubit system: L_S = 2, L_E = 2**(nqubit-1)
    Ls = 2
    Le = 2**(nqubit - 1)

    psi_t = np.asarray(psi_t).reshape((2,) * nqubit)

    # Move target site to axis 0, flatten the rest as environment
    axes = [site_a] + [i for i in range(nqubit) if i != site_a]
    psi_perm = np.transpose(psi_t, axes)
    psi_mat = psi_perm.reshape(Ls, Le)

    lam = np.sum(np.abs(psi_mat) ** 2, axis=0)

    # Keep only nonzero weights
    mask = lam > 1e-14
    lam = lam[mask]
    chi = (psi_mat[:, mask] / np.sqrt(lam)).T  # shape (n_env_nonzero, 2)

    # Return Bloch points and weights using your helper
    bloch_pts, weights = GQS_Bloch_Sphere_chi(chi, lam)
    return np.asarray(bloch_pts), np.asarray(weights)

gqs = [gqs_for_site(psi_t, site_a, nqubit) for psi_t in traj]
gqs_p = [gqs_for_site(psi_t, site_a, nqubit) for psi_t in traj_p]

print("Built GQS ensembles for each time step.")


In [ ]:
Wp = []
for (pts, w), (pts2, w2) in zip(gqs, gqs_p):
    try:
        W = Quantum_EMD(pts, w, pts2, w2)
    except Exception as e:
        W = np.nan
        if len(Wp) == 0:
            print("Wasserstein step failed (likely missing POT).")
            print("Error:", type(e).__name__, e)
    Wp.append(W)

Wp = np.asarray(Wp, dtype=float)
print("First few Wp:", Wp[:5])
print("Any NaNs?", np.isnan(Wp).any())


In [ ]:
t = np.arange(len(Wp))
plt.figure()
plt.plot(t, Wp, marker="o", linewidth=1)
plt.xlabel("time step (kick)")
plt.ylabel("Wasserstein distance (GQS)")
plt.title("Local GQS separation over time")
plt.show()
